In [1]:
import numpy as np
import pandas as pd 
from cl_cleaning import CleaningText as ct 
pd.set_option('float_format', '{:,.2f}'.format)
pd.set_option('display.max_columns', 70)

In [2]:
def drop_except(df, cols):
    df.drop(df.columns.difference(cols), axis=1, inplace=True)
    return df 

def get_antiguedad(celda):

              if (celda>=0) & (celda<16):
                     return'1.entre 0 y 15 dias'
              elif (celda>=16) & (celda<31):
                     return '2.entre 16 y 30 dias'
              elif (celda>=31) & (celda<61):
                     return '3.entre 31 y 60 dias'
              elif (celda>=61) & (celda<91):
                     return '4.entre 61 y 90 dias'
              elif (celda>=91) & (celda<181):
                     return '5.entre 91 y 180 dias'
              elif (celda>=181) & (celda<361):
                     return '6.entre 181 y 360 dias'
              elif celda>360:
                     return '7.mas de 360 dias'
              else:
                     return np.nan     

In [161]:
nc = pd.read_csv('input/planillas/210830_f11_angel.csv', sep=';', dtype='object')
nc = ct.norm_header(nc)

In [162]:
req_cols = ['nfolio', 'nsecuencia', 'prd_upc', 'xsubprod', 'qproducto',
  'dpactada_dt', 'propietario', 'dpactada_hd', 'xnombre_clt', 'xapellido_clt', 'xciudad_clt',
  'dcreacion', 'ddespacho', 'xobservacion','xestado', 'xservicio', 'borigen',
  'suc', 'tienda', 'costo_promedio', 'total_costo_promedio', 'fecha_corte', 
  'dias_creacion', 'dias_despacho', 'dias_descre', 'antiguedad_creacion', 'antiguedad_despacho',
  'grupo']

text_cols = ['xsubprod', 'qproducto', 'propietario', 'xnombre_clt', 'xapellido_clt', 'xciudad_clt',
 'xobservacion','xestado', 'xservicio', 'borigen', 'suc', 'tienda',  'grupo']

fnum_cols = ['nfolio', 'nsecuencia', 'prd_upc' ]

num_cols = ['costo_promedio', 'total_costo_promedio']

fechas = ['dcreacion','ddespacho', 'fecha_corte']

In [163]:
nc.fecha_creacion = pd.to_datetime(nc.fecha_creacion, format='%Y-%m-%d')
nc.dpactada_dt = pd.to_datetime(nc.dpactada_dt, format ='%Y-%m-%d')
nc.dpactada = pd.to_datetime(nc.dpactada, format ='%Y-%m-%d')
nc.loc[nc.fecha_despacho=='2608-08-26 00:00:00', 'fecha_despacho'] = np.nan
nc.fecha_despacho = pd.to_datetime(nc.fecha_despacho)

In [164]:
nc['fecha_corte'] = pd.to_datetime('today').normalize()

In [166]:
nc.loc[nc.fecha_despacho > '31/08/2021', 'fecha_despacho'] = np.nan

In [167]:
delta_creacion = nc.fecha_corte - nc.fecha_creacion 
nc['dias_creacion'] =  delta_creacion.dt.days

delta_despacho = nc.fecha_corte - nc.fecha_despacho 
nc['dias_despacho'] =  delta_despacho.dt.days

delta_descre =  nc.fecha_despacho - nc.fecha_creacion 
nc['dias_descre'] =  delta_descre.dt.days

nc['antiguedad_creacion'] = nc.dias_creacion.apply(get_antiguedad)
nc['antiguedad_despacho'] = nc.dias_despacho.apply(get_antiguedad)

In [168]:
nc = nc.rename(columns={'upc':'prd_upc', 'desc_sku':'xsubprod', 'cant_prod':'qproducto',
'dpactada':'dpactada_hd', 'nombre_clte':'xnombre_clt', 'apellido_clte':'xapellido_clt', 
'fecha_creacion':'dcreacion', 'estado':'xestado', 'servicio':'xservicio', 'origen':'borigen', 
'cod_local':'suc', 'nombre_local':'tienda','fecha_despacho':'ddespacho', 'observacion':'xobservacion',
'total_costo':'total_costo_promedio', 'ciudad_clte':'xciudad_clt'})

In [69]:
nc.to_csv('input/db-pbi/f11/210830-dfpap.csv', sep=';', index=False)

In [70]:
ncaux = nc.copy()

In [95]:
nc = ncaux.copy()

In [169]:
nc = drop_except(nc, req_cols)

In [170]:
print('Limpiando texto en columnas')
nc.loc[:, text_cols] = nc.loc[:, text_cols].apply(ct.clean_str)

Limpiando texto en columnas


In [171]:
print('Convirtiendo a número parte 1')
nc.loc[:,fnum_cols] = nc.loc[:,fnum_cols].apply(ct.clean_fnum)

Convirtiendo a número parte 1


In [172]:
nc.total_costo_promedio = pd.to_numeric(nc.total_costo_promedio)
nc.total_costo_promedio = round(nc.total_costo_promedio)
nc.total_costo_promedio = nc.total_costo_promedio.astype(int)

In [173]:
nc.costo_promedio = pd.to_numeric(nc.costo_promedio)
nc.costo_promedio = round(nc.costo_promedio)
nc.costo_promedio = nc.costo_promedio.fillna(0)
nc.costo_promedio = nc.costo_promedio.astype(int)

In [145]:
f11 = pd.read_csv('input/bases/210825_data_seguimiento_f11.csv', sep=';', dtype='object')

In [146]:
f11.ddespacho = f11.ddespacho.apply(lambda x: str(x).split(' ')[0])

In [147]:
f11.loc[f11.ddespacho=='17/02/3031', 'ddespacho'] = np.nan

In [149]:
f11.loc[f11.ddespacho=='0/01/1900', 'ddespacho'] = np.nan

In [150]:
f11.ddespacho = pd.to_datetime(f11.ddespacho, format='%d/%m/%Y')

In [151]:
f11.dcreacion = f11.dcreacion.apply(lambda x: str(x).split(' ')[0])

In [152]:
f11.dcreacion = pd.to_datetime(f11.dcreacion, format='%d/%m/%Y')

In [153]:
f11.fecha_corte = pd.to_datetime(f11.fecha_corte, format='%d/%m/%Y')

In [154]:
f11.loc[f11.ddespacho > '31/08/2021', 'ddespacho'] = np.nan

In [156]:
f11.loc[f11.ddespacho < '01/01/2000', 'ddespacho'] = np.nan

In [157]:

delta_creacion = f11.fecha_corte - f11.dcreacion 
f11['dias_creacion'] =  delta_creacion.dt.days

delta_despacho = f11.fecha_corte - f11.ddespacho 
f11['dias_despacho'] =  delta_despacho.dt.days

delta_descre =  f11.ddespacho - f11.dcreacion 
f11['dias_descre'] =  delta_descre.dt.days

f11['antiguedad_creacion'] = f11.dias_creacion.apply(get_antiguedad)
f11['antiguedad_despacho'] = f11.dias_despacho.apply(get_antiguedad)

In [158]:
f11 = drop_except(f11, req_cols)

In [174]:
f11save = pd.concat([f11, nc], axis=0)

In [175]:
f11save.to_csv('output/210831_data_seguimiento_f11.csv', sep=';', index=False) 